In [ ]:
import pandas
import skimage.exposure
import numpy
import mahotas
from aicsimageprocessing import read_ome_zarr
from aicsimageio import AICSImage

In [ ]:
def add_2D_shapes(csvname):

    def measure_2D_shapes(full_image,ProjType,feature_dict):
        # Channels 1,2,3 are dna, mem, structure intensities
        # Channels 4,5,6 are dna, mem, structure segmentations
        #channels = {0:'Brightfield',1:'DNA',2:'Membrane',3:'Structure'}
        compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
        features = {}
        desired_properties = ["label","area","perimeter","bbox","bbox_area","major_axis_length",
                "minor_axis_length","orientation","centroid","equivalent_diameter","extent",
                "eccentricity","convex_area","solidity","euler_number","moments","moments_central",
                "moments_normalized"]
        for eachcomp in compartments.keys():
            labels = skimage.measure.label(full_image[0,eachcomp,0,:,:])
            starting_props = ["area","perimeter","major_axis_length","minor_axis_length","eccentricity","orientation","centroid-1","centroid-0",
             "bbox_area","bbox-1","bbox-3","bbox-0","bbox-2","extent","solidity","euler_number","convex_area",
             "equivalent_diameter"]
            for x in range(4):
                for y in range(4):
                    starting_props.append(f"moments_normalized-{x}-{y}")
                    if x!= 3:
                        starting_props.append(f"moments_central-{x}-{y}")
            props=skimage.measure.regionprops_table(labels,properties=desired_properties)
            for feature_name in starting_props:
                object_name = "Shape_2D_{:}_{:}_{:}".format(compartments[eachcomp],ProjType, feature_name)
                if len(props[feature_name])==0:
                    feature_dict[object_name].append(numpy.nan)
                elif len(props[feature_name])==1:
                    feature_dict[object_name].append(float(props[feature_name]))
                else:
                    feature_dict[object_name].append(float(numpy.mean(props[feature_name])))
        return feature_dict

    cols2D = ['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
    #cols2D = ['max_projection_x']
    feature_dict={}
    starting_props = ["area","perimeter","major_axis_length","minor_axis_length","eccentricity","orientation","centroid-1","centroid-0",
     "bbox_area","bbox-1","bbox-3","bbox-0","bbox-2","extent","solidity","euler_number","convex_area",
     "equivalent_diameter"]
    for x in range(4):
        for y in range(4):
            starting_props.append(f"moments_normalized-{x}-{y}")
            if x!= 3:
                starting_props.append(f"moments_central-{x}-{y}")
                starting_props.append(f"moments_central-{x}-{y}")
    compartments = {4:'Nucleus',5:'Cell',6:'Organelle'}
    for eachcomp in compartments.keys():
        for feature_name in starting_props:
            for ProjType in cols2D:
                object_name = "Shape_2D_{:}_{:}_{:}".format(compartments[eachcomp],ProjType, feature_name)
                feature_dict[object_name]=[]
    
    df = pandas.read_csv(csvname)
    for _, eachrow in df.iterrows():
        for ProjType in cols2D:
            im = AICSImage(eachrow[ProjType]).data
            feature_dict = measure_2D_shapes(im,ProjType,feature_dict)
    import json
    with open("shape.json""w") as outfile:
        json.dump(feature_dict, outfile)
    df = pandas.concat([df,pandas.DataFrame(feature_dict)],axis=1)
    df.to_csv(csvname,index=False)

In [ ]:
add_2D_shapes('../data/mitocells_truncated.csv')